This generates all the code in the library

In [10]:
HEX = "0123456789ABCDEF" * 16

In [ ]:
# Hex
for i in range(16):
    print(f"/// This denotes the number {i}")
    print("#[derive(Clone, Copy, Debug)]")
    print(f"pub struct _{HEX[i]};")
    print()

for i in range(16):
    print(f"impl Hex for _{HEX[i]} {{ const NUMBER: u32 = {i}; }}")

In [ ]:
# Hex equal
print("/// This trait is the equal trait on hex numbers. False evaluates to _0 and true evaluates to _1")
print("pub trait HexEqual<H: Hex> { type Output: Binary; }")
for i in range(16):
    for j in range(16):
        if i == j:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _1; }}")
        else:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")

In [ ]:
# Hex assert eq
print("/// This trait is the equal trait on hex numbers with an assertion quality. False ones are simply not implemented")
print("pub trait HexAssertEqual<H: Hex> { }")
for i in range(16):
    print(f"impl HexAssertEqual<_{HEX[i]}> for _{HEX[i]} {{ }}")

In [ ]:
# Hex add 1
print("/// This is an internal implementation of addition of two number")
print("pub trait HexAdd1<H: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        print(f"impl HexAdd1<_{HEX[i]}> for _{HEX[j]} {{ type Output = _{HEX[i + j]}; type Carry = _{HEX[(i+j)//16]}; }}")

In [ ]:
# Hex add 3
print("/// This is an internal implementation of addition of three number")
print("pub trait HexAdd3<H1: Hex, H2: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        rs = f"_{HEX[i + j]}"
        cr = f"_{HEX[(i+j)//16]}"
        print(f"impl<H: Hex, R: Hex, C0: Hex, Cr: Hex, C_: Hex> HexAdd3<_{HEX[i]}, H> for _{HEX[j]} where H: HexAdd<{rs}, Output = R, Carry = C0>, C0: HexAdd<{cr}, Output = Cr, Carry = C_>, C_: HexAssertEqual<_0> {{ type Output = R; type Carry = Cr; }}")

In [ ]:
# Hex add 4 to 16

for adder in range(5, 17):
    h_where = ", ".join([f"H{i}: Hex" for i in range(1, adder)])
    print(f"/// This is an internal implementation of addition of {adder} number")
    print(f"pub trait HexAdd{adder}<{h_where}> {{ type Output: Hex; type Carry: Hex; }}")
    for i in range(16):
        h_generics = ", ".join([f"H{i}" for i in range(1, adder)])
        h_generics_bound = ", ".join([f"H{i}" for i in range(2, adder-1)])
        result = f"""
impl<{h_generics}, R, C1, C2, Result, Cr, C_> HexAdd{adder}<{h_generics}> for _{HEX[i]} where 
{h_where}, 
R: Hex, 
Result: Hex, 
C1: Hex, 
C2: Hex, 
Cr: Hex, 
C_: Hex, 
H1: HexAdd{adder-2}<{h_generics_bound}, Output = R, Carry = C1>, 
R: HexAdd3<_{HEX[i]}, H{adder-1}, Output = Result, Carry = C2>, 
C1: HexAdd<C2, Output = Cr, Carry = C_>, 
C_: HexAssertEqual<_0> 
{{ type Output = Result; type Carry = Cr; }}"""
        result = result.strip().replace("    ", "").replace("\n", "")
        print(result)
    print()

In [ ]:
# Hex complement
print("/// This represents 15 - x: the hexadecimal complement")
print("pub trait HexNot { type Output: Hex; }")
for i in range(16):
    print(f"""impl HexNot for _{HEX[i]} {{ type Output = _{HEX[15-i]}; }}""")

In [ ]:
# Hex multiply
print("/// This is an internal implementation of multiplication of two hexadecimal")
print("pub trait HexMul<H: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        rs = i * j
        mods = (i * j) // 16
        print(f"impl HexMul<_{HEX[i]}> for _{HEX[j]} {{ type Output = _{HEX[rs]}; type Carry = _{HEX[mods]}; }}")

In [ ]:
# Hex less than
print("/// This is an internal implementation of a < b")
print("pub trait HexLessThan<H: Hex> {type Output: Binary; }")
for i in range(16):
    for j in range(16):
        if i < j:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")
        elif i == j:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")
        else:
            print(f"impl HexLessThan<_{HEX[i]}> for _{HEX[j]} {{ type Output = _1; }}")

In [ ]:
# Other hex comparison

print("/// This is an internal implementation of a <= b")
print("pub trait HexLeq<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary, C: Binary> HexLeq<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, H: HexEqual<_{HEX[i]}, Output = B>, A: BinOr<B, Output = C> {{ type Output = C; }}""")

print("/// This is an internal implementation of a > b")
print("pub trait HexGreaterThan<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary, C: Binary> HexGreaterThan<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, H: HexEqual<_{HEX[i]}, Output = B>, A: BinNor<B, Output = C> {{ type Output = C; }}""")

print("/// This is an internal implementation of a >= b")
print("pub trait HexGeq<H: Hex> { type Output: Binary; }")
for i in range(16):
    print(f"""impl<H: Hex, A: Binary, B: Binary> HexGeq<H> for _{HEX[i]} where H: HexLessThan<_{HEX[i]}, Output = A>, A: BinNot<Output = B> {{ type Output = B; }}""")


In [ ]:
# Test cases for parsing
import random
random.seed(12345)
a = set()
while len(a) < 50:
    r = random.random() * 32
    r = int(2 ** r)
    if r in a:
        continue
    a.add(r)
for i, x in enumerate(sorted(a)):
    print(f"let a{i} = parse_integer!({x}); assert_eq!(a{i}.number(), {x});")

In [ ]:
# Test cases for addition
import random
random.seed(12345)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r + s >= 2 ** 32:
        continue
    a.add((r, s))
for i, (x, y) in enumerate(sorted(a, key=lambda x: x[0] + x[1])):
    print(f"let a{i}_0 = parse_integer!({x}); let a{i}_1 = parse_integer!({y}); let a{i}_2 = parse_integer!({x+y}); add_equal(a{i}_0, a{i}_1, a{i}_2);")

In [ ]:
# Test cases for subtraction
import random
random.seed(23456)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    a.add((r, s))
for i, (x, y) in enumerate(sorted(a, key=lambda x: x[1] - x[0])):
    print(f"let a{i}_0 = parse_integer!({x}); let a{i}_1 = parse_integer!({y}); let a{i}_2 = parse_integer!({y - x}); sub_equal(a{i}_0, a{i}_1, a{i}_2);")

In [ ]:
# Test cases for multiplication
import random
random.seed(34567)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    a.add((r, s))
for i, x in enumerate(sorted(a, key=lambda x: x[0] * x[1])):
    rs = x[0] * x[1]
    upper = rs // (2 ** 32)
    lower = rs % (2 ** 32)
    result = f"""
        let a{i}_0 = parse_integer!({x[0]});
        let a{i}_1 = parse_integer!({x[1]});
        let a{i}_2 = parse_integer!({upper});
        let a{i}_3 = parse_integer!({lower});
        mul_equal(a{i}_0,a{i}_1, a{i}_2, a{i}_3);
    """
    print(result.strip().replace("    ", "").replace("\n", " "))

In [ ]:
# Test cases for multiplication
import random
random.seed(42069)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if r > s:
        r, s = s, r
    if len(a) == 48 and r != s:
        a.add((r, r))
        a.add((s, s))
    else:
        a.add((r, s))
for i, x in enumerate(sorted(a)):
    if x[0] == x[1]:
        result = f"""
            let a{i}_0 = parse_integer!({x[0]});
            let a{i}_1 = parse_integer!({x[1]});
            is_less(a{i}_0, a{i}_1, False);
            is_less(a{i}_1, a{i}_0, False);
        """
    else:
        result = f"""
            let a{i}_0 = parse_integer!({x[0]});
            let a{i}_1 = parse_integer!({x[1]});
            is_less(a{i}_0, a{i}_1, True);
            is_less(a{i}_1, a{i}_0, False);
        """
    print(result.strip().replace("    ", "").replace("\n", " "))

In [154]:
# Actual psuedocode for division
def long_division(H, K):
    quotient = 0

    h = H

    for j in range(7, -1, -1):
        while h >= 16 ** j * K:
            h -= 16 ** j * K
            quotient += 16 ** j

    remainder = h
    return quotient, remainder

# Implementation psuedocode for Division
def long_division(H, K):
    quotient = 0

    h = H

    for j in (7, 6, 5, 4, 3, 2, 1, 0):
        for x in (8, 4, 2, 1):
            minus_me = h >= 16 ** j * x * K
            h -= 16 ** j * x * K * minus_me
            quotient += 16 ** j * x * minus_me

    remainder = h
    return quotient, remainder

r = 862761
s = 512

assert long_division(r, s) == (r//s, r%s)

In [150]:
# Division
g = 1

print("/// Returns the Quotient of H/K for H: Div<K, Output: ...>")
print("/// Implementation detail: This is an expanded version of long division - it takes O(1) steps where the constant is about 1000")
print("pub trait Div<K: IsInteger> { type Output: IsInteger; type Remainder: IsInteger; }")

# Generics
print("impl< K: IsInteger, ")
for i in range(8):
    print(f"Hx{i}: Hex, ", end="")
print()

impls = []

# Implementation
for t0 in (
"TypedInteger<_0, _0, _0, _0, _0, _0, _0, _1>",
"TypedInteger<_0, _0, _0, _0, _0, _0, _1, _0>",
"TypedInteger<_0, _0, _0, _0, _0, _1, _0, _0>",
"TypedInteger<_0, _0, _0, _0, _1, _0, _0, _0>",
"TypedInteger<_0, _0, _0, _1, _0, _0, _0, _0>",
"TypedInteger<_0, _0, _1, _0, _0, _0, _0, _0>",
"TypedInteger<_0, _1, _0, _0, _0, _0, _0, _0>",
"TypedInteger<_1, _0, _0, _0, _0, _0, _0, _0>",
):
    for x in (8, 4, 2, 1):
        tt = t0.replace("_1", f"_{x}")
        if g == 1:
            impls.append(f"K: DivInner<TypedInteger<Hx0, Hx1, Hx2, Hx3, Hx4, Hx5, Hx6, Hx7>, Zero, {tt}, Hout = H{g}, Qout = Q{g}>, ")
        else:
            impls.append(f"K: DivInner<H{g-1}, Q{g-1}, {tt}, Hout = H{g}, Qout = Q{g}>, ")
        g += 1

for i in range(1, g):
    print(f"H{i}: IsInteger, Q{i}: IsInteger, ")
print("B: Binary")
print("> Div<K> for TypedInteger<Hx0, Hx1, Hx2, Hx3, Hx4, Hx5, Hx6, Hx7> where")
for x in impls:
    print(x)
print("K: TypedGreaterThan<TypedInteger<_0, _0, _0, _0, _0, _0, _0, _0>, Output = B>,")
print("B: AssertTrue,")
print(f"{{ type Output = Q{g-1}; type Remainder = H{g-1}; }}")

/// Returns the Quotient of H/K for H: Div<K, Output: ...>
/// Implementation detail: This is an expanded version of long division - it takes O(1) steps where the constant is about 1000
pub trait Div<K: IsInteger> { type Output: IsInteger; type Remainder: IsInteger; }
impl< K: IsInteger, 
Hx0: Hex, Hx1: Hex, Hx2: Hex, Hx3: Hex, Hx4: Hex, Hx5: Hex, Hx6: Hex, Hx7: Hex, 
H1: IsInteger, Q1: IsInteger, 
H2: IsInteger, Q2: IsInteger, 
H3: IsInteger, Q3: IsInteger, 
H4: IsInteger, Q4: IsInteger, 
H5: IsInteger, Q5: IsInteger, 
H6: IsInteger, Q6: IsInteger, 
H7: IsInteger, Q7: IsInteger, 
H8: IsInteger, Q8: IsInteger, 
H9: IsInteger, Q9: IsInteger, 
H10: IsInteger, Q10: IsInteger, 
H11: IsInteger, Q11: IsInteger, 
H12: IsInteger, Q12: IsInteger, 
H13: IsInteger, Q13: IsInteger, 
H14: IsInteger, Q14: IsInteger, 
H15: IsInteger, Q15: IsInteger, 
H16: IsInteger, Q16: IsInteger, 
H17: IsInteger, Q17: IsInteger, 
H18: IsInteger, Q18: IsInteger, 
H19: IsInteger, Q19: IsInteger, 
H20: IsInteger, Q20:

In [ ]:
import random
random.seed(520)
a = set()
while len(a) < 50:
    r = int(2 ** (random.random() * 32))
    s = int(2 ** (random.random() * 32))
    if (r, s) in a:
        continue
    if len(a) == 48 and r != s:
        a.add((r, r))
        a.add((s, s))
    else:
        a.add((r, s))
    a.add((r, s))
for i, x in enumerate(sorted(a, key=lambda x: (x[0] // x[1], x[0] % x[1]))):
    h = x[0]
    k = x[1]
    q, r = (h//k, h%k)
    result = f"""
        let h{i} = parse_integer!({h});
        let k{i} = parse_integer!({k});
        let q{i} = parse_integer!({q});
        let r{i} = parse_integer!({r});
        div_equal(h{i},k{i}, q{i}, r{i});
    """
    print(result.strip().replace("    ", "").replace("\n", " "))